# Theory
## Background: **M**arkov **C**hain **M**onte **C**arlo (MCMC)

Oftentimes probability distributions are difficult to analytically express or normalize. However, it is still possible to sample over the entire distribution given the ratio of the probabilities between two points. Starting at some initial point $\vec{q_i}$, we can randomly draw a new value $\vec{q_{try}}$ and define this ratio as the probability to accept $\vec{q_{try}}$ as the new starting point:
$$P_{\rm accept} = \min\left(1, \frac{P(\vec{q_{\rm try}})}{P(\vec{q_{i}})}\right)$$

When implementing this acceptance probability in code, we uniformly draw a random number $r$ between 0 and 1. If $r < P_{\rm accept}$, then the step is accepted and $\vec{q_{i+1}} = \vec{q_{\rm try}}$. Otherwise, the step is rejected and $\vec{q_{i+1}} = \vec{q_{i}}$.

However, for a large number of parameters, $P_{\rm accept}$ becomes small, which causes the MCMC to take a long time to converge. To fix this, we must modify our algorithm to keep $P_{\rm accept}$ as large as possible.

## **H**amiltonian **M**onte **C**arlo (HMC)

HMC works by treating the probability distribution as related to the energy. The main advantage of this definition is that Hamiltonian paths conserve energy, which implies that the probability is also conserved. This allows us to accept the new step for every try, preventing the chain from getting stuck where the acceptance rate would otherwise drop to zero. For systems with a large number of parameters, this is especially useful.

The Hamiltonian is then defined with position $q$, momentum $p$, and mass $m$ as follows:

$$H(q,p) = \frac{p^2}{2m} + U(q), \space U(q) = -\ln{P(q)}$$

If we begin with initial position and momentum $(q_i,p_i)$, then $p_{\rm try}$ will be selected with a Gaussian random draw with a mean of zero and variance equal to mass $m$. $q_{\rm try}$ and $p_{\rm try}$ can then be obtained by numerically integrating Hamilton's equations:

$$\dot{q} = \frac{\partial H}{\partial p}, \space \dot{p} = -\frac{\partial H}{\partial q}$$

$$\\\\ \\\\$$

## Leapfrog Integration

It should be noted that integrators do not perfectly conserve energy, so a symplectic integrator must be carefully selected to prevent any systematic drift in energy. The acceptance probability then becomes:

$$P_{\rm accept} = \min(1, e^{-\Delta H})$$

where $\Delta H$ is simply the difference in energy between $q_{\rm try}$ and the previous point. We specifically choose to implement the Leapfrog algorithm.

For a timestep $\Delta t$, begin by evaluating the momentum $p(t)$ at the next half-step $t+\frac{\Delta t}{2}$:
$$p_{n+1/2} = p_{n-1/2} - \frac{\Delta t}{2} \frac{\partial U}{\partial q}|_{q_n} $$

Then utilize this to evaluate position at integer timesteps:

$$q_{n+1} = q_n + \frac{1}{m}p_{n+1/2} \Delta t$$

The process then repeats for the desired number of steps, with $p$ updated each half-step and $q$ updated each integer step.